In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
#import os
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"


In [29]:
# Importing the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head(5)



,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [30]:

train = data.iloc[:, 3:13].values
label = data.iloc[:, 13].values

labelencoder = LabelEncoder()
train[:, 1] = labelencoder.fit_transform(train[:, 1])
train[:, 2] = labelencoder.fit_transform(train[:, 2])



onehotencoder = OneHotEncoder(categorical_features = [1])
train = onehotencoder.fit_transform(train).toarray()
train = train[:, 1:]



In [77]:
# Feature Scaling
sc = StandardScaler()
train = sc.fit_transform(train)
x_train, x_test, y_train, y_test = train_test_split(train, label, test_size = 0.2, random_state = 2)


In [31]:
#keras 
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def build_classifier(optimizer,init):
    classifier = Sequential()
    classifier.add(Dense(6,input_dim = 11, init = init, activation = 'relu', ))
    classifier.add(Dropout(0.1))
    classifier.add(Dense(6, init = init, activation = 'relu'))
    classifier.add(Dropout(0.1))
    classifier.add(Dense(1, init = init, activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

model = KerasClassifier(build_fn = build_classifier)
parameters = { 'batch_size': [5,10,20],
               'nb_epoch': [100,200,400],
               'optimizer': ['adam', 'rmsprop'],
               'init': ['normal','uniform']
                                                               }



#tuning for the best parameter
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)

grids = grid_search.fit(x_train, y_train)
best_parameters = grids.best_params_
best_accuracy = grid_search.best_score_

#summarize results
print("Best: %f using %s" % (best_accuracy, best_parameters))

C:\Users\Mission\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, input_dim=11, activation="relu", kernel_initializer="normal")`
  app.launch_new_instance()
C:\Users\Mission\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="normal")`
C:\Users\Mission\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/10
6400/6400 [==============================] - 1s - loss: 0.4835 - acc: 0.7905     
Epoch 2/10
6400/6400 [==============================] - 1s - loss: 0.4339 - acc: 0.7913     
Epoch 3/10
6400/6400 [==============================] - 1s - loss: 0.4277 - acc: 0.7913     
Epoch 4/10
6400/6400 [==============================] - 1s - loss: 0.4280 - acc: 0.7963     
Epoch 5/10
6400/6400 [==============================] - 1s - loss: 0.4255 - acc: 0.8264     
Epoch 6/10
3275/6400 [==============>...............] - ETA: 0s - loss: 0.4128 - acc: 0.8369

In [8]:
print("Best: %f using %s" % (best_accuracy, best_parameters))

Best: 0.837500 using {'nb_epoch': 200, 'optimizer': 'adam', 'init': 'normal', 'batch_size': 5}


In [82]:

model = Sequential()
model.add(Dense(6,input_dim = 11, init = 'normal', activation = 'relu' ))
model.add(Dropout(0.1))
model.add(Dense(6, init = 'normal', activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
 

model.fit(x_train,y_train,nb_epoch= 200,  batch_size =  5)
y_pred = model.predict_classes(x_test)
cm = confusion_matrix(y_test, y_pred)

pred_leave = model.predict(train)
#pred_leave = (pred_leave>0.5)
pred_leave = pred_leave.flatten().tolist()
Prob_leave = pd.DataFrame({'CustomerID':data['CustomerId'].values , 'Prob_leave':pred_leave})
Prob_leave.tail(10)



C:\Users\Mission\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, input_dim=11, activation="relu", kernel_initializer="normal")`
  app.launch_new_instance()
C:\Users\Mission\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="normal")`
C:\Users\Mission\Anaconda3\lib\site-packages\keras\models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/200
8000/8000 [==============================] - 1s - loss: 0.4714 - acc: 0.7924     
Epoch 2/200
8000/8000 [==============================] - 1s - loss: 0.4327 - acc: 0.8060     
Epoch 3/200
8000/8000 [==============================] - 1s - loss: 0.4294 - acc: 0.8244     
Epoch 4/200
8000/8000 [==============================] - 1s - loss: 0.4236 - acc: 0.8286     
Epoch 5/200
8000/8000 [==============================] - 1s - loss: 0.4193 - acc: 0.8251     
Epoch 6/200
8000/8000 [==============================] - 1s - loss: 0.4195 - acc: 0.8284     
Epoch 7/200
8000/8000 [==============================] - 1s - loss: 0.4144 - acc: 0.8293     
Epoch 8/200
8000/8000 [==============================] - 1s - loss: 0.4156 - acc: 0.8283     
Epoch 9/200
8000/8000 [==============================] - 1s - loss: 0.4161 - acc: 0.8286     
Epoch 10/200
8000/8000 [==============================] - 1s - loss: 0.4135 - acc: 0.8351     
Epoch 11/200
8000/8000 [==============================] - 1

,CustomerID,Prob_leave
9990,15798964,0.151463
9991,15769959,0.758097
9992,15657105,0.141972
9993,15569266,0.069703
9994,15719294,0.076927
9995,15606229,0.126466
9996,15569892,0.068785
9997,15584532,0.107825
9998,15682355,0.319579
9999,15628319,0.096567
